# report

In [13]:
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
from contract_select import contract_select

In [19]:
data= 'spy'
date= '20170912'
maxDTE: int = 30,
minDTE: int = 10,
maxDistance: float = 0.2,
minDistance: float = 0.08,
maxTheta: float = -0.0,
minTheta: float = -1.2,
contract_type: str = 'C'
moneyness = 'out'

contract_pd =  pd.read_csv(data+'_cleaned/'+data+'_eod_'+date[:6]+'.csv', 
                   index_col=['[QUOTE_DATE]','[EXPIRE_DATE]'], skipinitialspace=True)
contract_pd = contract_pd.drop(columns='Unnamed: 0')
contract_pd = contract_pd.iloc[contract_pd.index.get_level_values('[QUOTE_DATE]')==date[:4]+'-'+date[4:6]+'-'+date[6:]]
contract_pd = contract_pd.loc[(contract_pd['[DTE]']>=minDTE) & (contract_pd['[DTE]']<=maxDTE) 
                & (contract_pd['[STRIKE_DISTANCE_PCT]']>=minDistance) & (contract_pd['[STRIKE_DISTANCE_PCT]']<=maxDistance)]

In [153]:
contract_select(selct_type='main', date= '20190913', spy_spx='spx', contract_type='P', minAsk=0.2, minDistance = 0.08)[1]

[UNDERLYING_LAST]  [DTE]  [STRIKE]  [P_BID]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                
2019-09-13   2019-10-04               3007.01   21.0    2685.0     1.65   
             2019-10-07               3007.01   24.0    2700.0     2.06   
             2019-10-04               3007.01   21.0    2745.0     2.60   

                            [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                            
2019-09-13   2019-10-04        1.70   -0.02474    0.00036   0.43013   
             2019-10-07        2.10   -0.03014    0.00041   0.53869   
             2019-10-04        2.66   -0.04009    0.00053   0.64100   

                            [P_THETA]  [STRIKE_DISTANCE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                 
2019-09-13   2019-10-04      -0.22720              322.0   
             2019-10-07      -0.23610              307.0   
             2019-10-04      -0.30988              262.0   

                            [STRIKE_DISTANCE_PCT]     utility  
[QUOTE_DATE] [EXPIRE_DATE]                                     
2019-09-13   2019-10-04                     0.107  456.861567  
             2019-10-07                     0.102  472.912451  
             2019-10-04                     0.087  476.710775

In [154]:
contract_select(selct_type='hedge', date= '20190913', spy_spx='spx', contract_type='P', minAsk=0.2, minDistance = 0.08)[1]

[UNDERLYING_LAST]  [DTE]  [STRIKE]  [P_BID]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                
2019-09-13   2019-10-11                300.99   28.0     277.0     0.53   
             2019-10-11                300.99   28.0     276.0     0.49   
             2019-10-11                300.99   28.0     272.5     0.39   

                            [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                            
2019-09-13   2019-10-11        0.53   -0.07202    0.00838   0.11642   
             2019-10-11        0.50   -0.06632    0.00769   0.10950   
             2019-10-11        0.39   -0.05106    0.00590   0.08808   

                            [P_THETA]  [STRIKE_DISTANCE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                 
2019-09-13   2019-10-11      -0.04221               24.0   
             2019-10-11      -0.03955               25.0   
             2019-10-11      -0.03371               28.5   

                            [STRIKE_DISTANCE_PCT]      utility  
[QUOTE_DATE] [EXPIRE_DATE]                                      
2019-09-13   2019-10-11                     0.080 -1052.231711  
             2019-10-11                     0.083 -1004.506662  
             2019-10-11                     0.095  -945.338264

In [ ]:
import pandas as pd
import os

def greek_calc(
    position: pd.DataFrame = None,
    ) ->pd.DataFrame:
    
"""
return a dataframe containing the current greeks exposure

input:
position: current position
"""


def pnl_calc(
    position: pd.DataFrame = None,
    ) ->pd.DataFrame:
    
"""
return a dataframe containing the pnl comparing to last trading day

input:
position: current position
"""


In [161]:
df1=contract_select(selct_type='main', date= '20190913', spy_spx='spx', contract_type='P', minAsk=0.2, minDistance = 0.08)
df2=contract_select(selct_type='hedge', date= '20190913', spy_spx='spy', contract_type='P', minAsk=0.2, minDistance = 0.08)

In [196]:
import pandas as pd
import os

def hedge_calc(
    main: pd.DataFrame = None,
    hedge: pd.DataFrame = None,
    main_data: str = None,
    hedge_data: str = None
    ) ->pd.DataFrame:
    
    """
    return a dataframe containing the information of selected contrats
    
    input:
    main: a dataframe containing the main contract to hedge
    hedge: a dataframe containing the hedge contract to hedge
    main_data: type of contract, spx or spy
    hedge_data: type of contract, spx or spy
    """

    if main_data == 'spy':
        columns = main.filter(regex='DELTA').columns
        main[columns] = main[columns]/10
        # columns = main.filter(regex='GAMMA').columns
        # main[columns] = main[columns]/10
        # columns = main.filter(regex='THETA').columns
        # main[columns] = main[columns]*10
    if hedge_data == 'spy':
        columns = hedge.filter(regex='DELTA').columns
        hedge[columns] = hedge[columns]/10
        # columns = main.filter(regex='GAMMA').columns
        # hedge[columns] = hedge[columns]/10
        # columns = hedge.filter(regex='THETA').columns
        # hedge[columns] = hedge[columns]*10
    
    # Extract columns whose names contain the desired string
    desired_string = ['GAMMA', 'VEGA', 'THETA']
    main_filtered_columns = []
    hedge_filtered_columns = []
    for i in desired_string:
        main_filtered_columns.append([col for col in main.columns if i in col])
        hedge_filtered_columns.append([col for col in main.columns if i in col])
        
    #flatten the column list
    main_filtered_columns=sum(main_filtered_columns,[])
    hedge_filtered_columns= sum(hedge_filtered_columns,[])
    main_mean = main[main_filtered_columns].values.mean(axis=0)
    main_mean_target = main_mean
    main_mean_target[-1] = 0
    
    def fun(x):
        x = np.array(x)
        utility = np.absolute(x.dot(hedge[hedge_filtered_columns].values)) - main_mean_target
        #add weights to the residual greeks after hedging 
        utility = utility * np.array([[50],[1],[10]])
        r=np.sum(utility)
        return r
    
    def fun1(x):
        x = np.array(x)
        utility = np.absolute(x.dot(hedge[hedge_filtered_columns].values)) - main_mean_target
        utility1 = utility * np.array([[1e2],[1],[10000]])
        return utility

    constraint1 = LinearConstraint(np.array(hedge[hedge_filtered_columns[-1]].values), lb=-main_mean[-1]*0.05, ub=np.inf)
    constraint2 = NonlinearConstraint(lambda x: np.sum(np.absolute(x)), lb=0, ub=1)
    constraint3 = LinearConstraint(np.array(hedge[hedge_filtered_columns[0]].values), lb=-abs(main_mean_target[0])*0.2, ub=abs(main_mean_target[0])*0.2)
    constraint4 = LinearConstraint(np.array(hedge[hedge_filtered_columns[1]].values), lb=-abs(main_mean_target[1])*0.2, ub=abs(main_mean_target[1])*0.2)
    
    x0 = [-0.1,-0.1,0.1]
    result = minimize(fun, x0, method='trust-constr', options={'maxiter':1000}, constraints={constraint1, constraint2, constraint3, constraint4}, tol=1e-4)
    hedge['weight'] = result.x
    main['weight'] = -1/len(main)
    
    x=result.x
    
    return main,result,fun1(x),hedge,np.sum(np.absolute(x)),x.sum()

In [197]:
x=np.array([ 0.00762753, -0.42346308,  0.04095649])

In [231]:
x=['[UNDERLYING_LAST]', '[DTE]', '[STRIKE]']
np.array(x).flatten()

array(['[UNDERLYING_LAST]', '[DTE]', '[STRIKE]'], dtype='<U17')

In [230]:
df1[1].columns

Index(['[UNDERLYING_LAST]', '[DTE]', '[STRIKE]', '[P_BID]', '[P_ASK]',
       '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]', '[STRIKE_DISTANCE]',
       '[STRIKE_DISTANCE_PCT]', 'utility', 'weight'],
      dtype='object')

In [200]:
hedge_calc(df1[1],df2[1],df1[0][:3],df2[0][:3])

(                            [UNDERLYING_LAST]  [DTE]  [STRIKE]  [P_BID]  \
 [QUOTE_DATE] [EXPIRE_DATE]                                                
 2019-09-13   2019-10-04               3007.01   21.0    2685.0     1.65   
              2019-10-07               3007.01   24.0    2700.0     2.06   
              2019-10-04               3007.01   21.0    2745.0     2.60   
 
                             [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
 [QUOTE_DATE] [EXPIRE_DATE]                                            
 2019-09-13   2019-10-04        1.70   -0.02474    0.00036   0.43013   
              2019-10-07        2.10   -0.03014    0.00041   0.53869   
              2019-10-04        2.66   -0.04009    0.00053   0.64100   
 
                             [P_THETA]  [STRIKE_DISTANCE]  \
 [QUOTE_DATE] [EXPIRE_DATE]                                 
 2019-09-13   2019-10-04      -0.22720              322.0   
              2019-10-07      -0.23610              307.0   
            

In [140]:
def fun(x):
    x = np.array(x)
    utility = np.absolute(x.dot(np.array([[0.008, 1.07542,-0.37072],[0.00071,0.98295,-0.34664],[0.0007,1.04561,-0.36438]]))-np.array([.0004,0.51869,0]))
    utility1 = utility \
        * np.array([[50],[1],[10]])
    r=np.sum(utility1)*10
    return r

def fun1(x):
    x = np.array(x)
    utility = np.absolute(x.dot(np.array([[0.008, 1.07542,-0.37072],[0.00071,0.98295,-0.34664],[0.0007,1.04561,-0.36438]]))-np.array([.0004,0.51869,0]))
    utility1 = utility * np.array([[1e2],[1],[10000]])
    return utility

constraint1 = LinearConstraint(np.array([-0.37072, -0.34664, -0.36438]), lb=-0.05, ub=0.05)
constraint2 = NonlinearConstraint(lambda x: np.sum(np.absolute(x)), lb=0, ub=1)
constraint3 = LinearConstraint(np.array([0.008, 0.00071, 0.0007]), lb=0.0004-0.0004*0.5, ub=0.0004+0.004*0.5)
constraint4 = LinearConstraint(np.array([1.07542, 0.98295, 1.04561]), lb=0.57869-0.57869*0.5, ub=0.57869+0.57869*0.5)

x0 = [-0.1,0.1,0.1]
result = minimize(fun, x0, method='trust-constr', options={'maxiter':1000}, constraints={constraint1,constraint2,constraint3,constraint4}, tol=1e-4)

In [143]:
x=result.x
fun1(x)

array([0.00360171, 0.24083277, 0.09349773])